In [1]:
import cudf as pd
# import cupy as cp
import numpy as np

In [2]:
adv_data = pd.read_csv('data/adv_data.csv')
users_data = pd.read_csv('data/users_data.csv')

In [3]:
adv_data.drop(['campaign_start_dtm', 'campaign_end_dtm', 'utm_campaign'], axis=1, inplace=True)

In [4]:
usdrub = 90.41

In [5]:
def convert_campaign_cost_to_rub(row):
    x = row['campaign_cost']
    if row['currency'] == 'USD':
        return x * usdrub
    else:
        return x

In [6]:
adv_data['campaign_cost'] = adv_data.apply(
    convert_campaign_cost_to_rub, axis=1)

In [7]:
adv_data.drop('currency', inplace=True, axis=1)

In [8]:
users_data = users_data[~users_data['user_id'].isna()]

In [9]:
adv_data['campaign_cost'].fillna(0.0, inplace=True)

In [10]:
users_data.drop('delete_dtm', axis=1, inplace=True)

In [11]:
user_phones = users_data['user_phone'].to_numpy()
user_test = np.zeros_like(user_phones)

In [12]:
for i, val in enumerate(user_phones):
    user_test[i] = str(val).strip('+')

In [13]:
users_data['user_phone'] = user_test

In [14]:
del user_test

In [15]:
users_uuids = users_data['user_id'].unique()

In [16]:
adv_data = adv_data[adv_data['user_id'].isin(users_uuids)]

In [17]:
adv_data.head(2)

,user_id,campaign_type,source_medium,interface,campaign_cost
0,db00f9c699edacc50090172cfaa3b43a,show,finance.rambler.ru / cpm,app,0.0
1,88aa22bca4fa1d6da73deb4b5c5ee8c0,show,yandex / organic,web,0.0


In [18]:
del users_uuids, users_data

In [19]:
adv_data_get_dummies = pd.get_dummies(adv_data, columns=[
                                                      'campaign_type', 'source_medium', 'interface'], dtype='int')

In [20]:
del adv_data

In [21]:
adv_data_get_dummies.head()

,user_id,campaign_cost,campaign_type_click,campaign_type_conversion,campaign_type_show,source_medium_android / mobile_app,source_medium_bing / organic,source_medium_email / email,source_medium_finance.rambler.ru / cpm,source_medium_google / organic,...,source_medium_vk / cpc,source_medium_web.telegram.org / referral,source_medium_ya.ru / cpc,source_medium_yahoo / organic,source_medium_yandex / cpc,source_medium_yandex / organic,source_medium_zen.yandex.ru / cpm,interface_api,interface_app,interface_web
0,db00f9c699edacc50090172cfaa3b43a,0.0000,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,88aa22bca4fa1d6da73deb4b5c5ee8c0,0.0000,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,e9e12347e50732804d2f8ded00743a07,0.0000,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,fa0cd363d1978a27a078152cd263c670,0.0000,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,fa0cd363d1978a27a078152cd263c670,121.1494,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [22]:
adv_data_get_dummies.columns

Index(['user_id', 'campaign_cost', 'campaign_type_click',
       'campaign_type_conversion', 'campaign_type_show',
       'source_medium_android / mobile_app', 'source_medium_bing / organic',
       'source_medium_email / email', 'source_medium_finance.rambler.ru / cpm',
       'source_medium_google / organic', 'source_medium_ios / mobile_app',
       'source_medium_not-set / none', 'source_medium_pikabu.ru / cpm',
       'source_medium_regular / email', 'source_medium_regular / sms',
       'source_medium_ru.android.bankproduct / mobile_app',
       'source_medium_ru.android.bankproduct_rustore / mobile_app',
       'source_medium_trigger / sms', 'source_medium_vk / cpc',
       'source_medium_web.telegram.org / referral ',
       'source_medium_ya.ru / cpc', 'source_medium_yahoo / organic',
       'source_medium_yandex / cpc', 'source_medium_yandex / organic',
       'source_medium_zen.yandex.ru / cpm', 'interface_api', 'interface_app',
       'interface_web'],
      dtype='object')

In [42]:
ad_campaign_type = adv_data_get_dummies.loc[adv_data_get_dummies['source_medium_zen.yandex.ru / cpm'] > 0]
ad_campaign_type.to_csv('data/adv_data_separated/zen_yandex_ru_cpm.csv')
del ad_campaign_type